# Example Data process

In [14]:
import torch
import pandas as pd
import numpy as np
import global_resources as gr
import os


device = gr.set_device()
print(f"Current training device: {device.capitalize()}.")

# GLOBAL VARIABLES
DTYPE = torch.float64
START_LEARNING_RATE = torch.tensor([0.01], device = device, dtype = DTYPE)


# Example Data process
# Read data
data_path = os.path.join(gr.default_dir, r'Data\breast-cancer-wisconsin.data')
df = gr.read_and_return_pd_df(data_path)

df.replace('?', np.nan, inplace = True)
df.dropna(inplace = True)
# df.replace('?', -99999, inplace = True)
df.drop(['id'], axis = 1, inplace = True)
df["bare_nuclei"] = df["bare_nuclei"].astype(np.int64)
df.dropna(inplace = True)

X = np.array(df.drop(['class'], axis = 1)).astype('float32')
y = np.array(df['class']).astype('float32')

y = np.where(y == 4, 1, np.where(y == 2, -1, y))

X_gpu = torch.tensor(X, dtype = DTYPE, device = device)
y_gpu = torch.tensor(y, dtype = torch.int64, device = device)

Current training device: Cuda.
Reading files from: D:\ImportanFiles\Coding Related\Repositories\Quantitative-Investment-Algorithms\Data\breast-cancer-wisconsin.data


In [15]:
import SVM.SVC as svc

# shuffle_tensor_row_wise
(
    X: torch.Tensor):

In [16]:
print(X.shape)

(683, 9)


In [17]:
rtn_value = svc.shuffle_tensor_row_wise(X_gpu)
print(rtn_value)
print(X)

tensor([[ 8., 10., 10.,  ...,  3.,  9.,  1.],
        [ 2.,  3.,  4.,  ...,  2.,  5.,  1.],
        [ 5.,  2.,  1.,  ...,  3.,  1.,  1.],
        ...,
        [10.,  6.,  4.,  ...,  9., 10.,  1.],
        [ 5.,  4.,  6.,  ...,  8., 10.,  1.],
        [ 3.,  3.,  5.,  ...,  7.,  1.,  1.]], device='cuda:0',
       dtype=torch.float64)
[[ 5.  1.  1. ...  3.  1.  1.]
 [ 5.  4.  4. ...  3.  2.  1.]
 [ 3.  1.  1. ...  3.  1.  1.]
 ...
 [ 5. 10. 10. ...  8. 10.  2.]
 [ 4.  8.  6. ... 10.  6.  1.]
 [ 4.  8.  8. ... 10.  4.  1.]]


# create_random_weights_bias
(
    X: torch.Tensor, 
    ) -> torch.Tensor:

In [18]:
weights_bias = svc.create_random_weights_bias(X_gpu)
print(weights_bias)

Creating random weights and bias with dtype: torch.float64
tensor([0.7816, 0.9310, 0.4268, 0.6187, 0.8644, 0.1691, 0.3488, 0.9605, 0.5587,
        0.0000], device='cuda:0', dtype=torch.float64)


# cal_distances
(
    X: torch.Tensor, 
    weights_and_bias: torch.Tensor,
    ) -> torch.Tensor:

In [19]:
distances = svc.cal_distances(X_gpu, weights_bias)
print(distances.dtype)
print(distances.shape)

torch.float64
torch.Size([683])


# hinge_loss
(
    distances: torch.Tensor, 
    labels: torch.Tensor, 
    margin: float = 1.0
    ) -> torch.Tensor:

In [20]:
losses = svc.hinge_loss(distances, y_gpu, margin = 1.0)
print(losses)

tensor([6.8602], device='cuda:0', dtype=torch.float64)


# hinge_loss_l2_panalty
(
    distances: torch.Tensor, 
    weights_and_bias: torch.Tensor,
    labels: torch.Tensor, 
    margin: float = 1.0, 
    l2_reg: float = 1e-4
    ) -> torch.Tensor:

In [21]:
losses_l2 = svc.hinge_loss_l2_panalty(distances, weights_bias, y_gpu)
print(losses_l2)

tensor([6.8604], device='cuda:0', dtype=torch.float64)


# update_model
(
    X: torch.Tensor, 
    y: torch.Tensor, 
    weights_and_bias: torch.Tensor,
    learning_rate: float = 0.01, 
    l2_penalty: bool = False,
    ) -> tuple[float, torch.Tensor, torch.Tensor]:

In [22]:
# for i in range(5000):
weights_bias.requires_grad_(True)
print(weights_bias.grad)
loss, weights, bias = svc.update_model(X_gpu, y_gpu, weights_bias, START_LEARNING_RATE)
print(weights_bias.grad)
print(loss)
print(weights)
print(bias)

None
tensor([1.9268, 0.8492, 0.9195, 0.8755, 1.3704, 0.8755, 1.3543, 0.8199, 0.6925,
        0.6501], device='cuda:0', dtype=torch.float64)
6.860155287487368
tensor([0.7623, 0.9225, 0.4176, 0.6100, 0.8507, 0.1604, 0.3352, 0.9523, 0.5518],
       device='cuda:0', dtype=torch.float64, grad_fn=<AsStridedBackward0>)
tensor(-0.0065, device='cuda:0', dtype=torch.float64,
       grad_fn=<AsStridedBackward0>)


# train
(X: torch.Tensor, 
          y: torch.Tensor, 
          weights_and_bias: torch.Tensor, 
          num_epochs = NUM_EPOCHS, 
          start_learning_rate = LEARNING_RATE, 
          l2_penalty = False, 
          print_every = int(100), 
          delta_loss_breaker = DELTA_LOSS_BREAKER, 
          patience = 10,
          norm_output_or_not = False
          ) -> tuple[torch.Tensor, torch.Tensor]:

## Parameters preparing

In [ ]:
X_gpu = torch.tensor(X, dtype = DTYPE, device = device)
y_gpu = torch.tensor(y, dtype = DTYPE, device = device)
weights_bias = svc.create_random_weights_bias(X = X_gpu)
lr_cpu = 0.01
START_LEARNING_RATE = torch.tensor([lr_cpu], device = device, dtype = DTYPE)
DELTA_LOSS_BREAKER = 3e-6
NUM_EPOCHS = int(1e5)
patience = 20

Creating random weights and bias with dtype: torch.float64


: 

In [24]:
weights, bias = svc.train(
    X = X_gpu, 
    y = y_gpu, 
    num_epochs = NUM_EPOCHS, 
    delta_loss_breaker = DELTA_LOSS_BREAKER, 
    l2_penalty = False, 
    weights_and_bias = weights_bias, 
    start_learning_rate = START_LEARNING_RATE,
    patience = patience)

Training with loss function: hinge loss.
Epoch 0 | Loss: 5.308304974008774 | Delta loss: None
Epoch 1 | Loss: 5.187315325806994 | Delta loss: 0.12098964820178004


Epoch 100 | Loss: 0.1929838252352103 | Delta loss: 0.0008803581588386067
Delta Loss smaller than threshold: Epoch 199 | Loss: 0.12438815254943038 | Delta loss: -2.25036021506142e-06
Epoch 200 | Loss: 0.1238241779909751 | Delta loss: 0.0005639745584552769
Epoch 300 | Loss: 0.10198654365120935 | Delta loss: 6.39398346198028e-05
Epoch 400 | Loss: 0.09592655487748143 | Delta loss: 2.4002327861344375e-05
Epoch 500 | Loss: 0.09373479778993506 | Delta loss: 1.9936484626142947e-05
Epoch 600 | Loss: 0.09175297855545006 | Delta loss: 1.7047074094472814e-05
Delta Loss smaller than threshold: Epoch 622 | Loss: 0.091364321837481 | Delta loss: 1.5953617726049973e-06
Delta Loss smaller than threshold: Epoch 692 | Loss: 0.09015293118809314 | Delta loss: 2.773919313289741e-06
Delta Loss smaller than threshold: Epoch 693 | Loss: 0.0901513518718714 | Delta loss: 1.5793162217447865e-06
Delta Loss smaller than threshold: Epoch 696 | Loss: 0.0900862171487933 | Delta loss: -2.1018390653354535e-06
Epoch 700 |

: 

: 

In [ ]:
SVC = svc.extract_svc(weights, bias)
unnormed = svc.extract_svc(weights, bias, norm = False)

print(unnormed)
print(SVC)

tensor([ 0.0764,  0.0801,  0.1083,  0.0632, -0.0205,  0.1678,  0.0646,  0.0835,
         0.0821, -2.3962], device='cuda:0', dtype=torch.float64,
       grad_fn=<CatBackward0>)
tensor([ 0.2798,  0.2937,  0.3969,  0.2316, -0.0751,  0.6151,  0.2369,  0.3059,
         0.3010, -8.7819], device='cuda:0', dtype=torch.float64,
       grad_fn=<DivBackward0>)


In [ ]:
accuracy = svc.score(weights, bias, X_gpu, y_gpu)
print(accuracy)

97.07174231332357
